In [1]:
import numpy as np
import re

from astropy.table import Table

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot
from pol_analysis.getStokes import GetStokes

In [2]:
def run_all(obj, band, star_coords, stddev_0_vals=None, bhd_coords=(1021.5, 70.), show_seeing_plots=False, pdata_force_new=False, phot_force_new=False, only_seeing_fit=False, centroid_box_size=5, pdata_filenames_for_masks=None, r_ap=1.0):

    #Star coords for PSF measurements
    star_ex_ref, star_ey_ref = star_coords

    #BHD coords
    bhd_ex_ref, bhd_ey_ref = bhd_coords

    #Create the data object. This will run the mask creation and the cosmic ray rejection.  
    bhd = PolData(obj, band, filenames_for_mask_creation=pdata_filenames_for_masks, force_new=pdata_force_new)

    #Create the photometry object and run the common steps. 
    phot = RunPhot(bhd, force_new=phot_force_new)
    phot.subtract_background(box_size=(25,25))
    phot.get_source_positions(box_size=centroid_box_size)

    #Now, go through all the OB pairs.
    #Now, go through all the OB pairs.
    ob_combs = list()
    for ob_pair in bhd.ob_pairs:
        ob_combs.append([[ob_pair[0]],[ob_pair[1]]])

    cato = open("{}_pol_{}.dat".format(obj,  band), "w")
    for k, ob_comb in enumerate(ob_combs):
        ob_ids = ob_comb[0]
        mjds = ob_comb[1]

        #Get the seeing. 
        fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
        if stddev_0_vals is not None:
            stddev_0 = stddev_0_vals[k]
        else:
            stddev_0 = np.ones(len(fnames))*1.1
        phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=show_seeing_plots, ob_ids=ob_ids, mjds=mjds)
        #print(phot.seeing)
        if only_seeing_fit:
            print(phot.seeing)
            continue

        #Get the photometry convolving to a common PSF. 
        phot.get_phot(r_ap=r_ap, force_new=True, apply_convolution=True, ob_ids=ob_ids, mjds=mjds)

        #Measure the stokes and polarization parameters. 
        stk_obj = GetStokes(bhd)
        stk_obj.get_pol(ob_ids=ob_ids, mjds=mjds)

        #Get the polarization values for the BHD. 
        epos_ref = np.loadtxt("{}/{}".format(bhd.phot_folder, re.sub(".fits", ".epos", fnames[0])))
        dist2 = (epos_ref[:,0]-bhd_ex_ref)**2 + (epos_ref[:,1]-bhd_ey_ref)**2
        k = np.argmin(dist2)
        #print("{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k]))

        str = "{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k])
        str += " {:5.1f} {:5.1f} {:5.1f}".format(np.min(phot.seeing), np.mean(phot.seeing), np.max(phot.seeing))
        if ob_ids is None:
            str += " All"
        else:
            for i, ob_id in enumerate(ob_ids):
                str += " {:10d} {:10d}".format(ob_id, mjds[i])
        print(str)
        cato.write(str+"\n")
    cato.close()



In [3]:
 #Load the log. 
log = Table.read("../proc/log.fits")

#Separate the standards from the science targets. The standards all have the same program ID. 
stds_log = log[log['Program_ID']=='60.A-9203(E)']

#Get the target names. 
obj_names = np.unique(stds_log['Target'])

#Set the reference location to find them. They should all be around the sample place. 
ex_ref = 1022.5
ey_ref = 70.

In [4]:
cond = (stds_log["Ret Angle"]==0)
aux_log = stds_log['Target','Filter','MJD_folder'][cond]
aux_log.sort('Target')
aux_log

Target,Filter,MJD_folder
bytes11,bytes9,int64
BD-12 5133,R_SPECIAL,59134
BD-12 5133,R_SPECIAL,59134
BD-12 5133,v_HIGH,60148
BD-14 4922,I_BESS,60238
Vela1,I_BESS,60238
Vela1,I_BESS,60238
Vela1,R_SPECIAL,60238
WD 0310-688,R_SPECIAL,59117
WD 1344+106,I_BESS,60148


## BD-12 5133

In [ ]:
obj_name = "BD-12 5133"

#### $R_{\rm SPECIAL}$

In [ ]:
band = "R_SPECIAL"
aux = PolData("W0116-0505",bband=band)
filenames_for_masks = aux.list_of_filenames(ob_ids=[2886768],mjds=[59135])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=1.5)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)

#### $v_{\rm HIGH}$

In [ ]:
band = "v_HIGH"
aux = PolData("W0116-0505",bband="I_BESS")
filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60146])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=1.5)#, show_seeing_plots=True, only_seeing_fit=True, pdata_force_new=True, phot_force_new=True)

## BD-14 4922

In [ ]:
obj_name = "BD-14 4922"

#### $I_{\rm BESS}$

Skip, images are completely out of focus.

In [ ]:
# band = "I_BESS"
# aux = PolData("W0116-0505",bband="I_BESS")
# filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60201])

# run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks)#, show_seeing_plots=True, only_seeing_fit=True, pdata_force_new=True, phot_force_new=True)

## Vela 1

In [5]:
obj_name = "Vela1"

#### $R_{\rm SPECIAL}$


In [6]:
band = "R_SPECIAL"
aux = PolData("W0116-0505",bband=band)
filenames_for_masks = aux.list_of_filenames(ob_ids=[2886768],mjds=[59135])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=1.5)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)


mkdir: work: File exists


Finding sources for image science_reduced_img.Vela1.60238.R_SPECIAL.chip1.1.fits


 7.79  0.06  171.9    0.2   0.8   0.9   1.0  200277985      60238


#### $I_{\rm BESS}$

In [ ]:
band = "I_BESS"
aux = PolData("W0116-0505",bband="I_BESS")
filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60201])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=2.5)#, show_seeing_plots=True, only_seeing_fit=True, pdata_force_new=True, phot_force_new=True)

## WD 0310-688

In [ ]:
obj_name = "WD 0310-688"

#### $R_{\rm SPECIAL}$

In [ ]:
band = "R_SPECIAL"
aux = PolData("W0116-0505",bband=band)
filenames_for_masks = aux.list_of_filenames(ob_ids=[2886768],mjds=[59135])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=2.5)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)

## WD 1344+106

In [ ]:
obj_name = "WD 1344+106"

#### $v_{\rm HIGH}$

In [ ]:
band = "v_HIGH"
aux = PolData("W0116-0505",bband="I_BESS")
filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60146])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=1.5)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)


#### $I_{\rm BESS}$

In [ ]:
band = "I_BESS"
aux = PolData("W0116-0505",bband="I_BESS")
filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60146])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=1.5)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)

## WD 2039-202

In [ ]:
obj_name = "WD 2039-202"

#### $R_{\rm SPECIAL}$

In [ ]:
band = "R_SPECIAL"
aux = PolData("W0116-0505",bband=band)
filenames_for_masks = aux.list_of_filenames(ob_ids=[2886768],mjds=[59135])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=2.0)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)

## WD 2359-434

In [ ]:
obj_name = "WD 2359-434"

#### $I_{\rm BESS}$

In [ ]:
band = "I_BESS"
aux = PolData("W0116-0505",bband="I_BESS")
filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60201])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=1.5)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)

#### $R_{\rm SPECIAL}$

In [ ]:
band = "R_SPECIAL"
aux = PolData("W0831+0140",bband=band)
filenames_for_masks = aux.list_of_filenames(ob_ids=[3565639],mjds=[60291])

run_all(obj_name, band, star_coords=(ex_ref, ey_ref), pdata_filenames_for_masks=filenames_for_masks, r_ap=2.0)#, show_seeing_plots=True, only_seeing_fit=True)#, pdata_force_new=True, phot_force_new=True)